In [1]:

import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gym
import gym_2048
from tqdm import tqdm

In [2]:
class GambleEnvironment():
    state0 = np.array([0])
    state1 = np.array([1])

    def reset(self):
        return self.state0
    
    def step(self, action):
        # lose always
        if action == 0:
            return self.state1, 1, True, None
        if action == 1:
            if np.random.random() <= 0.5:
                return self.state1, -1, True, None
            return self.state1, 1, True, None
        return self.state1, 1, True, None

In [3]:
replay_size = 10000

In [4]:
env = gym.make('2048-v0')

c:\Users\iguti\.virtualenvs\yannik_scribble-sRH0UPe6\lib\site-packages\gym\utils\passive_env_checker.py:20: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: int32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
c:\Users\iguti\.virtualenvs\yannik_scribble-sRH0UPe6\lib\site-packages\gym\utils\passive_env_checker.py:25: UserWarning: WARN: It seems a Box observation space is an image but the upper and lower bounds are not in [0, 255]. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(


In [5]:
envs = [gym.make('2048-v0') for _ in range(0,5)]

In [6]:
# MLP UDPRL stolen from https://github.com/BY571/Upside-Down-Reinforcement-Learning/blob/master/Upside-Down.ipynb

class BF(nn.Module):
    def __init__(self, state_space, action_space, hidden_size, seed):
        super(BF, self).__init__()
        torch.manual_seed(seed)
        self.actions = np.arange(action_space)
        self.action_space = action_space
        self.fc1 = nn.Linear(state_space, hidden_size)
        self.commands = nn.Linear(1, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, hidden_size)
        self.fc6 = nn.Linear(hidden_size, hidden_size)
        self.fc7 = nn.Linear(hidden_size, hidden_size)
        self.fc8 = nn.Linear(hidden_size, hidden_size)
        self.fc9 = nn.Linear(hidden_size, action_space)

        self.sigmoid = nn.Sigmoid()
        
    def forward(self, state, command):       
               
        out = self.sigmoid(self.fc1(state))
        command_out = self.sigmoid(self.commands(command))
        out = out * command_out
        out = torch.relu(self.fc2(out))
        out = torch.relu(self.fc3(out))
        out = torch.relu(self.fc4(out))
        out = torch.relu(self.fc5(out))
        out = torch.relu(self.fc6(out))
        out = self.fc9(out)
        
        return out
    
    def action(self, state, desire, horizon):
        """
        Samples the action based on their probability
        """
        command_win = desire
        command_lose = -1*desire
        action_prob_win = self.forward(state.expand(1, -1), command_win.expand(1, -1))[0,:]
        action_prob_lose = self.forward(state.expand(1, -1), command_lose.expand(1, -1))[0,:]
        probs = torch.softmax(action_prob_win - action_prob_lose, dim=-1)
        action = torch.distributions.categorical.Categorical(probs=probs).sample()
        return action

In [7]:
import random


class ReplayBuffer():
    def __init__(self, max_size):
        self.max_size = max_size
        self.buffer = []
        
        
    def add_sample(self, states, actions, rewards):
        episode = {"states": states, "actions":actions, "rewards": rewards, "summed_rewards":sum(rewards)}
        self.buffer.append(episode)

        self.buffer.append(episode)

    def get_nbest(self, n):
        self.sort()
        return self.buffer[:n]
        
    
    def sort(self):
        # keep the max buffer size
        self.buffer = self.buffer[:self.max_size]
    
    def get_random_samples(self, batch_size):
        self.sort()
        idxs = np.random.randint(0, len(self.buffer), batch_size)
        batch = [self.buffer[idx] for idx in idxs]
        return batch
    
    def __len__(self):
        return len(self.buffer)

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

bf = BF(256, 4, 256, 1).to(device)
#bf.load_state_dict(torch.load("model.pth"))
optimizer = optim.Adam(params=bf.parameters(), lr=1e-3)

# create replay buffer:
buffer = ReplayBuffer(replay_size)

In [9]:
init_desired_reward = 1
init_time_horizon = 2

def generate_episode(desired_return = torch.FloatTensor([init_desired_reward]), desired_time_horizon = torch.FloatTensor([init_time_horizon])):    
    """
    Generates more samples for the replay buffer.
    """
    next_state = env.reset()
    bf.eval()
    states = []
    actions = []
    rewards = []
    done = False
    while not done:
        states.append(next_state.flatten())
        action = bf.action(torch.from_numpy(next_state.flatten()).float().to(device), desired_return.to(device), desired_time_horizon.to(device))
        next_state, reward, done, info = env.step(action)
        # reward = 0
        # if done:
        #     reward = -1
        # if info['highest'] >= 16:
        #     reward = 1
        #     done = True
        desired_return -= reward
        actions.append(action)
        rewards.append(reward)
    return states, actions, rewards

In [13]:
init_desired_reward = 1
init_time_horizon = 2

def generate_episodes(desired_return = init_desired_reward):    
    """
    Generates more samples for the replay buffer.
    """
    states = []
    actions = []
    dones = []
    rewards = [0 for _ in envs]

    state_stack = []
    state_round = []
    done_round = [False for _ in envs]
    for i, game in enumerate(envs):
        s = game.reset().flatten()
        state_round.append(s)
        state_stack.append(torch.cat([torch.from_numpy(s).float().to(device), torch.tensor([desired_return]).to(device)]))
    states.append(state_round)
    dones.append(done_round)

    while not np.all(done_round):
        X = torch.stack(state_stack)
        stateNr = 256
        state = X[:,0:stateNr]
        d = X[:,stateNr:stateNr+1]
        a = torch.argmax(bf(state, d) - bf(state, -1*d), dim=1).cpu().detach().numpy()
        actions.append(a)

        state_stack = []
        state_round = []
        done_round = []
        for i, game in enumerate(envs):
            s, reward, done, info = game.step(a[i])
            s = s.flatten()
            state_round.append(s)
            state_stack.append(torch.cat([torch.from_numpy(s).float().to(device), torch.tensor([desired_return]).to(device)]))
            done_round.append(done)
            if done and rewards[i] == 0:
                reward = -1
                if info['highest'] >= 16:
                    reward = 1
                rewards[i] = reward
        states.append(state_round)
        dones.append(done_round)

    return states, actions, rewards
generate_episodes(torch.FloatTensor([init_desired_reward]))

([[array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
   array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 

In [14]:
# init replay buffer with random trajectories:
for i in tqdm(range(0, int(replay_size))):
    states, actions, rewards = generate_episode(torch.FloatTensor([1]))
    buffer.add_sample(states, actions, rewards)

  1%|▏         | 139/10000 [00:03<04:09, 39.57it/s]


KeyboardInterrupt: 

In [155]:
win = 0
for x in buffer.get_random_samples(10000):
    r = x['rewards']
    if r[len(r)-1] == 1:
        win += 1
    
print(win/10000)

0.4967


In [143]:
generate_episode(torch.FloatTensor([1]))

([array([0])], [tensor(0)], [1])

In [111]:
# FUNCTIONS FOR Sampling exploration commands
last_few = 1000
def sampling_exploration( top_X_eps = last_few):
    """
    This function calculates the new desired reward and new desired horizon based on the replay buffer.
    New desired horizon is calculted by the mean length of the best last X episodes. 
    New desired reward is sampled from a uniform distribution given the mean and the std calculated from the last best X performances.
    where X is the hyperparameter last_few.
    
    """
    
    top_X = buffer.get_nbest(last_few)
    #The exploratory desired horizon dh0 is set to the mean of the lengths of the selected episodes
    new_desired_horizon = np.mean([len(i["states"]) for i in top_X])
    # save all top_X cumulative returns in a list 
    returns = [i["summed_rewards"] for i in top_X]
    # from these returns calc the mean and std
    mean_returns = np.mean(returns)
    std_returns = np.std(returns)
    # sample desired reward from a uniform distribution given the mean and the std
    new_desired_reward = np.random.uniform(mean_returns, mean_returns+std_returns)

    return torch.FloatTensor([new_desired_reward])  , torch.FloatTensor([new_desired_horizon]) 

In [112]:


# FUNCTIONS FOR TRAINING
def select_time_steps(saved_episode):
    """
    Given a saved episode from the replay buffer this function samples random time steps (t1 and t2) in that episode:
    T = max time horizon in that episode
    Returns t1, t2 and T 
    """
    # Select times in the episode:
    T = len(saved_episode["states"]) # episode max horizon 
    t1 = np.random.randint(0,T)
    t2 = T

    return t1, t2, T

def create_training_input(episode, t1, t2):
    """
    Based on the selected episode and the given time steps this function returns 4 values:
    1. state at t1
    2. the desired reward: sum over all rewards from t1 to t2
    3. the time horizont: t2 -t1
    
    4. the target action taken at t1
    
    buffer episodes are build like [cumulative episode reward, states, actions, rewards]
    """
    state = episode["states"][t1] 
    desired_reward = sum(episode["rewards"][t1:t2])
    time_horizont = t2-t1
    action = episode["actions"][t1]
    return state, desired_reward, time_horizont, action

def create_training_examples(batch_size):
    """
    Creates a data set of training examples that can be used to create a data loader for training.
    ============================================================
    1. for the given batch_size episode idx are randomly selected
    2. based on these episodes t1 and t2 are samples for each selected episode 
    3. for the selected episode and sampled t1 and t2 trainings values are gathered
    ______________________________________________________________
    Output are two numpy arrays in the length of batch size:
    Input Array for the Behavior function - consisting of (state, desired_reward, time_horizon)
    Output Array with the taken actions 
    """
    input_array = []
    output_array = []
    # select randomly episodes from the buffer
    episodes = buffer.get_random_samples(batch_size)
    for ep in episodes:
        #select time stamps
        t1, t2, T = select_time_steps(ep)
        # For episodic tasks they set t2 to T:
        t2 = T
        state, desired_reward, time_horizont, action = create_training_input(ep, t1, t2)
        input_array.append(torch.cat([torch.FloatTensor(state), torch.FloatTensor([desired_reward]), torch.FloatTensor([time_horizont])]))
        output_array.append(action)
    return input_array, output_array

def train_behavior_function(batch_size):
    """
    Trains the BF with on a cross entropy loss were the inputs are the action probabilities based on the state and command.
    The targets are the actions appropriate to the states from the replay buffer.
    """
    bf.train()
    X, y = create_training_examples(batch_size)

    X = torch.stack(X)
    stateNr = 1
    state = X[:,0:stateNr]
    d = X[:,stateNr:stateNr+1]
    h = X[:,stateNr+1:stateNr+2]
    command = d
    y = torch.stack(y).long()
    optimizer.zero_grad()
    y_ = bf(state.to(device), command.to(device)).float()
    pred_loss = F.cross_entropy(y_, y)   
    pred_loss.backward()
    optimizer.step()
    return pred_loss.detach().cpu().numpy()



In [113]:
def run_loop():
    i = 0
    minLoss = 100
    minLossI = 0
    while True:
        # compute desired horizon and desired reward
        
        i += 1
        perc = 0
        for _ in range(0, int(replay_size/100)):
            goal = 1
            rew, hor = torch.FloatTensor([1]), torch.FloatTensor([1])
            # if np.random.random() <= 0.5:
            #     goal = -1
            #     rew, hor = torch.FloatTensor([-1]), torch.FloatTensor([1])
            states, actions, rewards = generate_episode(rew, hor)
            if rewards[len(rewards)-1] == goal:
                perc+=1
            #buffer.add_sample(states, actions, rewards)

        loss = train_behavior_function(int(replay_size/10))
        if loss < minLoss-0.0001:
            minLoss = loss
            minLossI = i
        if i - minLossI > 100:
            print("Early Stopping, No Improvment")
            return
        print(i, loss, minLoss, minLossI, perc/(replay_size/100), flush=True)

In [114]:
run_loop()
# SAVE MODEL
name = "model.pth"
torch.save(bf.state_dict(), name)

1 0.82680017 0.82680017 1 1.0
2 0.61505276 0.61505276 2 1.0
3 0.04295875 0.04295875 3 1.0
4 0.038360935 0.038360935 4 1.0
5 0.057843823 0.038360935 4 1.0
6 0.09147246 0.038360935 4 1.0
7 0.04207101 0.038360935 4 1.0
8 0.008094875 0.008094875 8 1.0
9 0.03397139 0.008094875 8 1.0
10 0.048192028 0.008094875 8 1.0
11 0.06072189 0.008094875 8 1.0
12 0.04590947 0.008094875 8 1.0
13 0.037430607 0.008094875 8 1.0
14 0.05224005 0.008094875 8 1.0
15 0.060480975 0.008094875 8 1.0
16 0.03849717 0.008094875 8 1.0
17 0.042795487 0.008094875 8 1.0
18 0.022580445 0.008094875 8 1.0
19 0.063020535 0.008094875 8 1.0
20 0.02657094 0.008094875 8 1.0


KeyboardInterrupt: 

In [116]:
state = env.reset()
goal = torch.FloatTensor([1])
F.softmax(bf(torch.FloatTensor(state.flatten()), goal) - bf(torch.FloatTensor(state.flatten()), -1*goal), dim=-1)

tensor([3.6677e-01, 1.3098e-04, 6.3310e-01], grad_fn=<SoftmaxBackward0>)

In [117]:
win = 0
for x in range(0, 100):
    _, _, r = generate_episode(torch.FloatTensor([1]))
    if r[len(r)-1] == 1:
        win += 1
    
print(win)

100
